<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week2/part3/02_WORKFLOWS_PARALELOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

¿Qué es Pydantic?

Pydantic es una librería de Python para validación de datos y gestión de configuraciones usando anotaciones de tipos de Python. Su lema es: "Data validation using Python type hints".

In [ ]:
%pip install -U langchain-google-genai pydantic langgraph
%pip install -U langgraph langchain-google-genai pydantic

In [ ]:
from google.colab import userdata
import os

api_key = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = api_key
print('API Key cargada:', 'Sí' if api_key else 'No')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Configure the Gemini API
MODEL_ID = os.getenv("GEMINI_MODEL", "models/gemini-2.5-flash-lite")

llm = ChatGoogleGenerativeAI(model=MODEL_ID, temperature=0.2)
print("✅ LLM listo:", MODEL_ID)

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# ============================================================================
# ESTADO DEL GRAFO
# ============================================================================
# Define la estructura de datos compartida entre todos los nodos
class Estado(TypedDict):
    tema: str              # Tema principal sobre el que se generará contenido
    chiste: str            # Chiste generado por el primer LLM
    historia: str          # Historia generada por el segundo LLM
    poema: str             # Poema generado por el tercer LLM
    salida_combinada: str  # Resultado final que combina todos los contenidos

In [ ]:
# ============================================================================
# NODOS - Funciones que ejecutan tareas específicas
# ============================================================================

def llamar_llm_1(estado: Estado):
    """
    Primera llamada al LLM: Genera un chiste
    - Recibe el tema del estado compartido
    - Retorna el chiste generado
    """
    mensaje = llm.invoke(f"Escribe un chiste sobre {estado['tema']}")
    return {"chiste": mensaje.text}

def llamar_llm_2(estado: Estado):
    """
    Segunda llamada al LLM: Genera una historia
    - Se ejecuta en paralelo con llamar_llm_1 y llamar_llm_3
    - Retorna la historia generada
    """
    mensaje = llm.invoke(f"Escribe una historia sobre {estado['tema']}")
    return {"historia": mensaje.text}

def llamar_llm_3(estado: Estado):
    """
    Tercera llamada al LLM: Genera un poema
    - También se ejecuta en paralelo
    - Retorna el poema generado
    """
    mensaje = llm.invoke(f"Escribe un poema sobre {estado['tema']}")
    return {"poema": mensaje.text}

def agregador(estado: Estado):
    """
    Combina todos los resultados en una sola salida formateada
    - Espera a que los 3 LLMs terminen (punto de sincronización)
    - Organiza el contenido en un formato legible
    """
    combinado = f"¡Aquí tienes una historia, un chiste y un poema sobre {estado['tema']}!\n\n"
    combinado += f"HISTORIA:\n{estado['historia']}\n\n"
    combinado += f"CHISTE:\n{estado['chiste']}\n\n"
    combinado += f"POEMA:\n{estado['poema']}"
    return {"salida_combinada": combinado}

In [ ]:
# ============================================================================
# CONSTRUCCIÓN DEL FLUJO DE TRABAJO PARALELO
# ============================================================================

# Crear el constructor del grafo con el esquema de Estado
constructor_paralelo = StateGraph(Estado)

# ----------------------------------------------------------------------------
# Agregar nodos al grafo
# ----------------------------------------------------------------------------
constructor_paralelo.add_node("llamar_llm_1", llamar_llm_1)
constructor_paralelo.add_node("llamar_llm_2", llamar_llm_2)
constructor_paralelo.add_node("llamar_llm_3", llamar_llm_3)
constructor_paralelo.add_node("agregador", agregador)

# ----------------------------------------------------------------------------
# Agregar conexiones (edges) entre nodos
# ----------------------------------------------------------------------------

# EJECUCIÓN PARALELA: Las 3 llamadas al LLM inician simultáneamente desde START
constructor_paralelo.add_edge(START, "llamar_llm_1")  # Rama 1: Generar chiste
constructor_paralelo.add_edge(START, "llamar_llm_2")  # Rama 2: Generar historia
constructor_paralelo.add_edge(START, "llamar_llm_3")  # Rama 3: Generar poema

# PUNTO DE SINCRONIZACIÓN: Las 3 ramas convergen en el agregador
# El agregador espera a que todos los LLMs terminen antes de ejecutarse
constructor_paralelo.add_edge("llamar_llm_1", "agregador")
constructor_paralelo.add_edge("llamar_llm_2", "agregador")
constructor_paralelo.add_edge("llamar_llm_3", "agregador")

# El agregador termina el proceso
constructor_paralelo.add_edge("agregador", END)

# ----------------------------------------------------------------------------
# Compilar el grafo en una cadena ejecutable
# ----------------------------------------------------------------------------
flujo_paralelo = constructor_paralelo.compile()


In [ ]:
# ============================================================================
# VISUALIZACIÓN
# ============================================================================

# Mostrar el diagrama del flujo de trabajo
display(Image(flujo_paralelo.get_graph().draw_mermaid_png()))

In [ ]:
# ============================================================================
# EJECUCIÓN
# ============================================================================

# Ejecutar el flujo con el tema "gatos"
estado = flujo_paralelo.invoke({"tema": "gatos"})

# Imprimir el resultado final combinado
print(estado["salida_combinada"])